In [6]:
import torch
import torch.nn as nn

import requests
from io import BytesIO
from PIL import Image, ImageOps

import numpy as np

In [25]:
# Define the image
url = "https://www.jsonline.com/gcdn/presto/2020/08/13/PMJS/086e6c1e-a091-494b-b940-6b3259589040-MJS_UW_MADISON_COVID_12894.JPG"
def get_image_from_url(url):
    response = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    img = Image.open(BytesIO(response.content))
    img = img.resize((600, 500))
    img = ImageOps.grayscale(img)
    return img

img = get_image_from_url(url)
img = torch.from_numpy(np.array(img)).float().unsqueeze(0).unsqueeze(0) / 255.

# Define Sobel kernels
sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32).view(1, 1, 3, 3)
sobel_y = torch.tensor([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=torch.float32).view(1, 1, 3, 3)

In [26]:
# Convolution operation using PyTorch's F.conv2d function
import torch.nn.functional as F
response_x = F.conv2d(img, sobel_x, padding=1)
response_y = F.conv2d(img, sobel_y, padding=1)
grad_magnitude = torch.sqrt(response_x**2 + response_y**2)

tensor([[[[-2.6980e+00, -3.6039e+00, -3.6078e+00,  ..., -2.2510e+00,
           -2.2510e+00, -1.6863e+00],
          [-1.9608e-02, -2.3530e-02, -2.7451e-02,  ..., -6.6667e-02,
           -5.8824e-02, -3.9216e-02],
          [-1.9608e-02, -2.3529e-02, -2.7451e-02,  ..., -2.3530e-02,
            3.9214e-03,  1.1765e-02],
          ...,
          [ 1.9608e-02,  3.5294e-02,  1.9608e-02,  ..., -1.0980e-01,
           -1.2157e-01, -8.2353e-02],
          [ 2.7059e-01,  2.0784e-01,  3.1372e-02,  ...,  2.9802e-08,
           -3.1373e-02, -4.3137e-02],
          [ 1.6118e+00,  2.1216e+00,  2.1373e+00,  ...,  1.8824e+00,
            1.8784e+00,  1.4039e+00]]]])

The highest values appear on the vertical edges for sobel_x, and horizontal edges for sobel_y. this is because they're designed to capture the intensity along these dimensions

there are no changes or edges to detect in that region.